<a href="https://colab.research.google.com/github/jithinmjoison/fast_ai_from_foundations/blob/main/Varitional_Autoencoders_July_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensforflow.keras.backend as K
from tensorflow.keras import (layers,  models, datasets, callbacks,losses,optimizers, metrics,)

from scipy.stats import norm

ModuleNotFoundError: No module named 'tensforflow'